In [2]:
from pypdf import PdfReader
import pandas as pd
import pdfplumber
import requests
from docx import Document
import json
import urllib.request
import cloudconvert
import numpy as np
import datetime
import os

In [ ]:
df = pd.DataFrame()

path = "D:\Programing\python\schedule\pdfs"
files = []

for f in os.listdir(path):
    if f.endswith('.pdf'):
        files.append(f)

def str_to_time(string):
    if string == "время":
        return string
    elif len(string) == 0:
        return string
    elif len(string) < 9:
        new_string = '0' + string
        output = new_string[0:2] + ":" + new_string[2:7] + ":" + new_string[7:9]
        return output
    else:
        output = string[0:2] + ":" + string[2:7] + ":" + string[7:9]
        return output
    
for file in files:
    with pdfplumber.open(file) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                
                df_table = pd.DataFrame(table, columns=['c1', 'c2','c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13', 'c14', 'c15', 'c16']).drop(['c3', 'c15', 'c16'], axis=1)
                df_table['c1'] = df_table['c1'].apply(lambda x: (''.join(str(x).splitlines())[::-1].title()) if str(x) != "None" else x)
                df_table.loc[0, 'c1'] = 'День недели'
                df_table['c1'] = df_table['c1'].ffill()
                df_table['c2'] = df_table['c2'].ffill()
                df_table['c4'] = df_table['c4'].apply(str_to_time)
                df_table['c2'] = df_table['c2'].apply(lambda x: (''.join(str(x).splitlines())[::-1].title()) if str(x) != "None" and str(x) != "дата" else x)
                
                #display(df_table)
                #print(df_table.shape)
                df_table.columns = [df_table.iloc[0]]
                #df_table.drop([0], inplace=True, axis=0)
                    
                if len(df) == 0:
                    df = pd.concat([df, df_table], ignore_index=False, sort=False)
                else:
                    df = pd.concat([df, df_table.iloc[:, 3:]], ignore_index=False, sort=False, axis=1)

    df.drop([0], inplace=True)
    #df.reset_index()
    #print(df.shape)
    df.to_excel(f'{file[:-4]}.xlsx')
    print(f"{file} - DONE!")
    df = pd.DataFrame()     

print('END!')

In [ ]:
df = pd.read_excel('ВО Расписание очное осень 2024-2025 уч.год. (16.12.2024-22.12.2024) четная неделя.xlsx')
df.reset_index()
df.dropna(how='all', inplace=True)
#display(df)

column_names = list(df.iloc[:, 4::2].columns)

group = 'МАб-231.2'
day = '16.12.24'
now_date = datetime.datetime.now().date().strftime("%d.%m.%Y")

group_df = df.iloc[:, [1,2,3, df.columns.get_loc(group), df.columns.get_loc(group)+1]]
mask = group_df['дата'] == day
output_df = group_df[mask]
output_df.fillna('', inplace=True)
display(output_df)

print(f'{output_df.iloc[0, 0]:11} | {output_df.iloc[0, 1]}\n')
#print('-'*10)
for i in range(len(output_df)):
    #print(f'{output_df.iloc[i, 2]:11} | {output_df.iloc[i, 3]:28} | {str(output_df.iloc[i, 4])[:-2]}')
    print(f'{i+1} | {output_df.iloc[i, 3]:35} | {str(output_df.iloc[i, 4])}')

In [ ]:
# ФУНКЦИЯ ЭМУЛЯЦИИ ПОИСКА РАСПИСАНИЯ НА СЕГОДНЯШНИЙ ДЕНЬ ПО ВВОДИМОЙ ГРУППЕ И ВИДЕ ОБУЧЕНИЯ

def day_classes(group, profile):

    path = "D:\Programing\python\schedule\pdfs\\"
    files = []

    # Сбор всех pdf в директории
    for f in os.listdir(path):
        if f.endswith('.xlsx') and profile in f and f[0] != '~':
            files.append(f)

    output_string = ''
    
    for f in files:
        try:
            df = pd.read_excel(path + f)
            df.reset_index()
            df.dropna(how='all', inplace=True)

            column_names = list(df.iloc[:, 4::2].columns)
            now_date = str(datetime.datetime.now().date().strftime("%d.%m.%y"))

            group_df = df.iloc[:, [1,2,3, df.columns.get_loc(group), df.columns.get_loc(group)+1]]
            #display(group_df)
            mask = group_df['дата'] == f'{now_date}'
            output_df = group_df[mask]
            output_df.fillna('', inplace=True)
            #display(output_df)

            if len(output_df) > 0:
                output_string += f'{output_df.iloc[0, 0]:11} | {output_df.iloc[0, 1]}\n'
                #print('-'*10)
                for i in range(len(output_df)):
                    #print(f'{output_df.iloc[i, 2]:11} | {output_df.iloc[i, 3]:28} | {str(output_df.iloc[i, 4])[:-2]}')
                    output_string += f'\n{i+1} | {output_df.iloc[i, 3]:35} | {str(output_df.iloc[i, 4])}'
                
                print(output_string)
                
        except KeyError:
            print('key err')
            continue

        except IndexError:
            print('index err')
            continue

day_classes('ГОт-211.2 / ГОтс-221.2', 'СПО')

In [ ]:
files = ['СПО, Лицей  Расписание  09.12.2024-15.12.2024 нечетная неделя.xlsx', 
         'СПО, Лицей  Расписание  23.12.2024-28.12.2024 нечетная неделя.xlsx']

for f in files:
    try:
        df = pd.read_excel(f)
        df.reset_index()
        df.dropna(how='all', inplace=True)
        #display(df)

        column_names = list(df.iloc[:, 4::2].columns)

        #group = 'МАб-231.2'
        group = 'ОПс-201.2'
        day = '16.12.24'
        now_date = datetime.datetime.now().date().strftime("%d.%m.%Y")

        print(df.columns.get_loc(group))
        
        group_df = df.iloc[:, [1,2,3, df.columns.get_loc(group), df.columns.get_loc(group)+1]]
        mask = group_df['дата'] == day
        output_df = group_df[mask]
        output_df.fillna('', inplace=True)
        display(output_df)
    
    except KeyError:
        continue

In [ ]:
file = '14.12.2024 Расписание занятий на субботы заочное 2024-25г.pdf'

df = pd.DataFrame()
with pdfplumber.open(file) as pdf:
    for page in pdf.pages:
        tables = page.extract_tables()
        for table in tables:
                
            df_table = pd.DataFrame(table, columns=['c1', 'c2','c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])
            #display(df_table)
            df_table['c2'] = df_table['c2'].apply(lambda x: (''.join(str(x).splitlines())[::-1].title()) if str(x) != "None" else x)
            df_table['c1'] = df_table['c1'].apply(lambda x: (''.join(str(x).splitlines()[::-1])) if str(x) != "None" and str(x) != "Дата" else x)            
            df_table.loc[0, 'c1'] = 'Дата'
            df_table.loc[0, 'c2'] = 'День недели'
            df_table['c1'] = df_table['c1'].ffill()
            df_table['c2'] = df_table['c2'].ffill()
            df_table['c3'] = df_table['c3'].apply(str_to_time)

                
            display(df_table)
            #print(df_table.shape)
            df_table.columns = [df_table.iloc[0]]
            #df_table.drop([0], inplace=True, axis=0)
                    
            if len(df) == 0:
                df = pd.concat([df, df_table], ignore_index=False, sort=False)
            else:
                df = pd.concat([df, df_table.iloc[:, 3:]], ignore_index=False, sort=False, axis=1)

df.drop([0], inplace=True)
#df.reset_index()
#print(df.shape)
df.to_excel(f'{file[:-4]}.xlsx')
print(f"{file} - DONE!")
df = pd.DataFrame()     